In [1]:
from ollama import chat, ChatResponse, Client
from tqdm import tqdm
import pandas as pd
import random
import json
import csv
import os
import re

In [2]:
annot = ["SolutionExp", "HintExp", "AlgoExp", "TimeComExp", "FixingExp", "TestExp", "QuestionExp", "SubQuestion", "TestQuestion", "Irrelevant"]

general_prompt = str(
    f"Your are given a problem statement and a comment. Classify the comment into one of the following classes: "
        f"SolutionExp: A detailed comment explaining how to solve a problem, more like a step-by-step guide. Often it contains the whole code solution"
        f"HintExp: It can be viewed as a sparse variant of SolutionExp in which various elements from an entire solution are presented"
        f"AlgoExp: A comment which states mere the algorithm used with little to no information"
        f"TimeComExp: A comment which states mere the complexity little to no information"
        f"FixingExp: A comment which explains or hints how to solve a specific bug in an implementation"
        f"TestExp: A comment which explains the result of a test case"
        f"QuestionExp: A relevant question about a specific task in the problem"
        f"SubQuestion: A comment which asks insights about why a submission fails"
        f"TestQuestion: A comment asking for insights about a specific test’s result"
        f"Irrelevant: Any other comment that does not fit the above labels"
    )

In [3]:
pb_set = pd.read_csv('problemset.csv')
disc = pd.read_csv('discussion.csv', low_memory=False)

precleaning the dataframe from readerbench

In [4]:
df = pd.read_csv('annotated_labels.csv')
new_df = df[['desc', 'sentiment', 'text']]
new_df.to_csv('annotated_labels.csv', index=True)

generating annotations process

In [5]:
correct, predicted = list(new_df['sentiment']), []

In [6]:
def annotate(client: Client):
    global correct, predicted
    for _, row in new_df.iterrows():
        # Comment prompting
        comment = "This is the comment: " + row["text"]
        prompt = (general_prompt + comment + "give me in ONLY ONE WORD the prediction for the class")

        try:
            response = client.generate(model="llama3.1:70b", prompt=prompt, system="You are an educational expert.")
            predicted.append(response['response'])
        except Exception as e:
            print(e)

In [7]:
if __name__ == "__main__":
    token = open('token.txt').readline()
    cl = Client(
        host='https://chat.readerbench.com/ollama',
        headers={"Authorization": f"Bearer {token}"}
    )

    annotate(cl)


confusion matrix

In [19]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true=correct, y_pred=predicted, labels=annot)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=annot)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix', fontsize=15, pad=20)
plt.xlabel('Prediction', fontsize=11)
plt.ylabel('Actual', fontsize=11)
#Customizations
plt.gca().xaxis.set_label_position('top')
plt.gca().xaxis.tick_top()
plt.gca().figure.subplots_adjust(bottom=0.2)
plt.gca().figure.text(0.5, 0.05, 'Prediction', ha='center', fontsize=13)
plt.savefig('confusion_matrix1.png', dpi=300, format='png')
plt.close()

enhanced annotation

In [54]:
def annotate2():
    global correct, predicted
    for idx, row in new_df.iterrows():
        # Comment prompting
        comment = "This is the comment: " + row["text"]
        prompt = (general_prompt + comment + "give me in ONLY ONE WORD the prediction for the class")


        try:
            cerinta = pb_set.loc[pb_set['Description'] == row['desc'], 'Problem Name'].iloc[0]

            problem_lines = pb_set[pb_set['Description'].str.contains(row['desc'], regex=False)] # row['desc'] is actually the comment
            line = problem_lines[problem_lines['Description'] == row['desc']]
            print(idx, cerinta, line['Description'])
            # if line['Parent']
        except Exception as e:
            print(e)
        if idx == 6:
            break

In [55]:
annotate2()

0 last-person-to-fit-in-the-bus
 908    <p>Table: <code>Queue</code></p>\n\n<pre>+----...
Name: Description, dtype: object
1 remove-sub-folders-from-the-filesystem
 925    <p>Given a list of folders <code>folder</code>...
Name: Description, dtype: object
2 cat-and-mouse-ii
 1312    <p>A game is played by a cat and a mouse named...
Name: Description, dtype: object
3 number-of-ways-to-stay-in-the-same-place-after-some-steps
 948    <p>You have a pointer at index <code>0</code> ...
Name: Description, dtype: object
4 find-the-duplicate-number
 229    <p>Given an array of integers <code>nums</code...
Name: Description, dtype: object
5 coin-change-ii
 400    <p>You are given an integer array <code>coins<...
Name: Description, dtype: object
6 flip-columns-for-maximum-number-of-equal-rows
 831    <p>You are given an <code>m x n</code> binary ...
Name: Description, dtype: object
